In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")


In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# Async

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")

config = {"configurable": {"thread_id": "abc123"}}


In [ ]:
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver

# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
aapp = workflow.compile(checkpointer=MemorySaver())



In [ ]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]

# Async invocation:
output = await aapp.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


In [ ]:

query = "What's my name?"
input_messages = [HumanMessage(query)]

# Async invocation:
output = await aapp.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# Adding a System Message

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# Trimming

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")

In [ ]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages


trimmer = trim_messages(
    max_tokens=6_500_000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed_messages})
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
import aiosqlite


db_path = 'checkpoints.db'
conn = sqlite3.connect(db_path, check_same_thread=False)

#memory = SqliteSaver(conn)
# Does not work
#memory = AsyncSqliteSaver(conn)


memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
        
print(type(app))

In [ ]:
from pprint import pprint

config = {"configurable": {"thread_id": "abc12"}}
inputs = {"messages": [HumanMessage(content="What is 1+1?")]}

async with AsyncSqliteSaver.from_conn_string(db_path) as saver:
    graph = workflow.compile(checkpointer=saver)
    async for event in graph.astream_events(inputs, config, version="v1"):
        pprint(event)

In [ ]:
# THIS COULD BE THE ONE I NEED

from pprint import pprint

config = {"configurable": {"thread_id": "abc12"}}
inputs = {"messages": [HumanMessage(content="Hi! I'm Bob.")]}

async with AsyncSqliteSaver.from_conn_string(db_path) as saver:
    graph = workflow.compile(checkpointer=saver)        
    async for chunk, metadata in graph.astream(
        inputs,
        config,
        stream_mode="messages",
    ):
        if isinstance(chunk, AIMessage):  # Filter to just model responses
            print(chunk.content, end="")

In [ ]:
import json


config = {"configurable": {"thread_id":  "abc12"}}



with SqliteSaver.from_conn_string(db_path) as checkpointer:
    graph = workflow.compile(checkpointer=checkpointer)
    state_history = graph.get_state_history(config) 
    values = next(state_history).values  
    print(values) 
    print(len(values['messages']))
    for message in values['messages']:
        if isinstance(message, HumanMessage):
            print(f"Human: {message.content}")
        else:
            print(f"AI:    {message.content}")
    
    #state_history_list = list(state_history)
    #print(state_history_list)
    # for state in state_history:
    #     print(state)
    #     # print(f"Step: {state.step}")
    #     # print(f"Created At: {state.created_at}")
    #     # print(f"Values: {state.values}")
    #     # print("-----")
 
    #list(graph.get_state_history(config))

In [ ]:
query = "What is my name"
input_messages = [HumanMessage(query)]

async for chunk, metadata in app.astream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

In [ ]:
import asyncio

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import StateGraph

builder = StateGraph(int)
builder.add_node("add_one", lambda x: x + 1)
builder.set_entry_point("add_one")
builder.set_finish_point("add_one")
async with AsyncSqliteSaver.from_conn_string("checkpoints.db") as memory:
    graph = builder.compile(checkpointer=memory)
    coro = graph.ainvoke(1, {"configurable": {"thread_id": "thread-1"}})
    print(asyncio.run(coro))

# Async history

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import START, MessagesState, StateGraph



system_prompt = "Answer all questions to the best of your ability. Answer concisely but correctly. If you do not know the answer, just say 'I don’t know.'"
model = init_chat_model("deepseek-r1:32b", model_provider="ollama")

trimmer = trim_messages(
    max_tokens=6_500_000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

graph = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed_messages})
    response = model.invoke(prompt)
    return {"messages": [response]}

graph.add_edge(START, "model")
graph.add_node("model", call_model)



In [ ]:
print("IN get_chat_interactions_count")
interactions_count = 0
db_path="/Users/dan/Code/LLM/DeepDive/backend/resources/checkpoints.db"
chat_id="ea9123ff-9a8e-46c0-a53f-22b8f88e3202"

with SqliteSaver.from_conn_string(db_path) as checkpointer:            
    config = {"configurable": {"thread_id": chat_id}}
    print("before compiled_graph")
    compiled_graph = graph.compile(checkpointer=checkpointer)
    print("after compiled_graph")
    state_history = compiled_graph.get_state_history(config) 
    print("after state_history")
    last_interaction = next(state_history, None)
    print("after last_interaction")
    if last_interaction:
        values = last_interaction.values  
        if 'messages' in values:
            interactions_count = len(values['messages'])
            
interactions_count

In [ ]:
import sys
import os
sys.path.append('/Users/dan/Code/LLM/DeepDive/backend')

import app.app_session
file_path = app.app_session.__file__
parent_directory = os.path.dirname(file_path)
grandparent_directory = os.path.dirname(parent_directory)
grandparent_directory

# GET CONVERSATION HISTORY


In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import START, MessagesState, StateGraph

model = init_chat_model("llama3.2:1b", model_provider="ollama")
system_prompt = "Do your best to answer correctly"

def create_graph():
    #TODO THIS DOES NOT LOOK RIGHT    
    trimmer = trim_messages(
        max_tokens=6_500_000,
        strategy="last",
        token_counter=model,
        include_system=True,
        allow_partial=False,
        start_on="human",
    )

    prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    graph = StateGraph(state_schema=MessagesState)

    def call_model(state: MessagesState):
        trimmed_messages = trimmer.invoke(state["messages"])
        prompt = prompt_template.invoke({"messages": trimmed_messages})
        response = model.invoke(prompt)
        return {"messages": [response]}

    graph.add_edge(START, "model")
    graph.add_node("model", call_model)

    return graph


graph = create_graph()
        

In [2]:

db_path = "/Users/dan/Code/LLM/DeepDive/backend/resources/checkpoints.db"

def get_chat_interactions_count(chat_id):
    interactions_count = 0
    with SqliteSaver.from_conn_string(db_path) as checkpointer:            
        config = {"configurable": {"thread_id": chat_id}}
        compiled_graph = graph.compile(checkpointer=checkpointer)
        state_history = compiled_graph.get_state_history(config) 
        last_interaction = next(state_history, None)
        if last_interaction:
            values = last_interaction.values  
            if 'messages' in values:
                interactions_count = len(values['messages'])
    return interactions_count

In [3]:
def get_chat_interactions(chat_id):
    interactions=[]
    with SqliteSaver.from_conn_string(db_path) as checkpointer:            
        config = {"configurable": {"thread_id": chat_id}}
        compiled_graph = graph.compile(checkpointer=checkpointer)
        state_history = compiled_graph.get_state_history(config) 
        last_interaction = next(state_history, None)
        if last_interaction:
            values = last_interaction.values  
            if 'messages' in values:
                for message in values['messages']:
                    #print(type(message))
                    #print(message.content)
                    interaction_type = 'user' if isinstance(message,HumanMessage) else 'ai'
                    interaction = {'type':interaction_type, 'content':message.content}
                    interactions.append(interaction)
    return interactions

In [4]:
chat_id="ea9123ff-9a8e-46c0-a53f-22b8f88e3202"
get_chat_interactions(chat_id)


[{'type': 'user', 'content': 'hello'},
 {'type': 'ai',
  'content': '<think>\nOkay, so I\'m trying to figure out what "hello" means in this context. The user greeted me with "hello," and the assistant responded by asking how they can assist today. It seems like a standard interaction where someone starts a conversation.\n\nI wonder if there\'s more to it. Maybe the user is testing the AI or just initiating a chat. Since the response was pretty straightforward, I don\'t think there\'s any hidden meaning here. The assistant is prompting for further assistance, so my next step should be to provide whatever help they need based on their follow-up question.\n\nI guess I should make sure that in my responses, I\'m clear and helpful without overcomplicating things. Since the user said "hello," it might be a way to get the conversation started, so I need to be ready for any questions or topics they bring up next.\n</think>\n\nThe user greeted with "hello," which is a standard way to start a co